In [23]:
from selenium import webdriver
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")

# URL of the webpage containing the dropdown menu
url = "https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season=2023-24"
driver.get(url)

In [24]:
years = driver.find_elements_by_class_name("DropDown_select__4pIg9")[0].text.split('\n')
table_data = []

for i in range(len(years[:5])):
    year = years[i]
    
    # Find the season dropdown menu
    season_dropdown_menu = Select(driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[1]/div/div/div[1]/label/div/select')) 
    season_dropdown_menu.select_by_visible_text(year)
    
    # Introduce a wait to allow the page to load
    time.sleep(5)
    
    # Wait for the page dropdown to be present
    page_dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select'))
    )

    # Find the page dropdown element using its HTML attribute class name
    page_dropdown = Select(driver.find_elements_by_class_name("DropDown_select__4pIg9")[5])

    # Change the selected option by index, value 'All'
    page_dropdown.select_by_visible_text('All')
    
    # Find the table on the page
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table'))
    )
    
    # Store the table data in a variable for looping
    tbody = table.find_element(By.TAG_NAME, 'tbody')
    rows = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
    )
    
    for row in rows:
        try:
            row_cells = row.find_elements(By.TAG_NAME, 'td')
            row_text = [cell.text for cell in row_cells]
            row_text.append(year)
            table_data.append(row_text) 
            print(f"Adding data for player #{row_text[0]} {row_text[1]} for the {year} season")
        except:
            # Handle StaleElementReferenceException by refinding the row
            rows = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
            )
# Get table headers
headers = table.find_element_by_xpath('//thead').text.split(' ')
headers.append('YEAR')

# Close the browser window
driver.quit()  

Adding data for player #1 Joel Embiid for the 2023-24 season
Adding data for player #2 Kevin Durant for the 2023-24 season
Adding data for player #3 Luka Doncic for the 2023-24 season
Adding data for player #4 Shai Gilgeous-Alexander for the 2023-24 season
Adding data for player #5 Giannis Antetokounmpo for the 2023-24 season
Adding data for player #6 Stephen Curry for the 2023-24 season
Adding data for player #7 Nikola Jokic for the 2023-24 season
Adding data for player #8 Jayson Tatum for the 2023-24 season
Adding data for player #9 Anthony Edwards for the 2023-24 season
Adding data for player #10 Trae Young for the 2023-24 season
Adding data for player #11 Tyrese Maxey for the 2023-24 season
Adding data for player #12 Damian Lillard for the 2023-24 season
Adding data for player #13 LeBron James for the 2023-24 season
Adding data for player #14 Paul George for the 2023-24 season
Adding data for player #15 Tyrese Haliburton for the 2023-24 season
Adding data for player #16 Jalen Bruns

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=119.0.2151.58)


In [25]:
import pandas as pd

# Getting the headers from the data array
# It is important to remove them from the data array afterwards in order to be parsed correctly by Pandas
df = pd.DataFrame(table_data, columns=headers)
df


,#,PLAYER,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,YEAR
0,2023-24,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1,Joel Embiid,PHI,15,34.7,32.1,10.4,21.1,49.4,1.0,...,88.1,3.2,8.1,11.3,6.3,1.0,1.9,3.7,36.9,2023-24
2,2,Kevin Durant,PHX,15,36.9,31.4,10.7,20.1,53.3,2.3,...,89.1,0.3,6.7,7.1,5.5,0.7,1.2,4.0,31.5,2023-24
3,3,Luka Doncic,DAL,16,35.1,30.5,10.5,21.6,48.7,3.8,...,76.5,0.6,7.3,7.9,7.8,1.3,0.4,3.9,31.3,2023-24
4,4,Shai Gilgeous-Alexander,OKC,15,34.3,30.4,11.1,20.7,53.4,1.3,...,93.7,0.9,5.3,6.1,6.2,2.3,0.9,2.3,33.5,2023-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4374,129,Udonis Haslem,MIA,75,24.0,7.3,2.7,6.0,45.9,0.0,...,76.5,2.5,3.8,6.3,0.7,0.4,0.3,1.0,10.3,2003-04
4375,130,Derek Fisher,LAL,82,21.5,7.1,2.5,7.0,35.2,0.6,...,79.7,0.4,1.5,1.9,2.3,1.3,0.0,1.0,6.6,2003-04
4376,131,Damon Jones,MIL,82,24.6,7.0,2.6,6.4,40.1,1.2,...,76.4,0.2,1.9,2.1,5.8,0.4,0.0,1.3,10.0,2003-04
4377,132,Raul Lopez,UTA,82,19.7,7.0,2.7,6.3,43.1,0.3,...,86.3,0.3,1.6,1.9,3.7,0.8,0.0,2.1,7.5,2003-04


In [26]:
file_path = 'data.csv'
df.to_csv(file_path, index=False)